In [ ]:
# come to https://github.com/wildoctopus/huggingface-cloth-segmentation

# 得到原始图片文件夹1，分割mask掩码图片文件夹2。现在合并文件夹1和文件夹2.

In [6]:
import os
import numpy as np
from PIL import Image, ImageEnhance
from tqdm import tqdm

def apply_mask_to_image(image_path, mask_path, output_path, brightness_factor=1.5):
    image = Image.open(image_path).convert('RGB')
    mask = Image.open(mask_path).convert('RGB')  # Open mask as RGB to handle different colors

    # Apply mask to the image
    image_np = np.array(image)
    mask_np = np.array(mask)

    # Separate mask channels
    red, green, blue = mask_np[:,:,0], mask_np[:,:,1], mask_np[:,:,2]
    combined_mask = np.maximum.reduce([red, green, blue])  # Combine different colors into one mask

    masked_image_np = np.multiply(image_np, combined_mask[:,:,None] / 255.0)
    masked_image = Image.fromarray(masked_image_np.astype(np.uint8))

    # Enhance brightness
    enhancer = ImageEnhance.Brightness(masked_image)
    bright_image = enhancer.enhance(brightness_factor)

    # Save the brightened image
    bright_image.save(output_path)

def merge_images_with_masks(image_folder, mask_folder, output_folder, brightness_factor=1.5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = os.listdir(image_folder)
    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(image_folder, image_file)
        mask_path = os.path.join(mask_folder, image_file)

        if os.path.exists(mask_path):
            output_path = os.path.join(output_folder, image_file)
            apply_mask_to_image(image_path, mask_path, output_path, brightness_factor)
        # else:
            # print(f"Mask not found for {image_file}")

# Define the paths to the folders
image_folder = '/data1/dxw_data/llm/redbook_final/script_next/data_img_20%'
mask_folder = '/data1/dxw_data/llm/redbook_final/script_next/data_img_20%_segcloth_background'
output_folder = '/data1/dxw_data/llm/redbook_final/script_next/data_img_20%_segcloth_background_combineall'

# Merge images with masks
merge_images_with_masks(image_folder, mask_folder, output_folder)


Processing images: 100%|██████████| 52853/52853 [30:16<00:00, 29.09it/s]  
